In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Step 1: Get the HTML content of the website

* `response = requests.get(url)`: 获取网页数据
* `response.encoding = 'gbk'`: 设置编码格式（中文）
* `soup = BeautifulSoup(...)`: 把网页传给Beatiful Soup

* `print(soup.prettify())`: 清理网页并查看格式

In [2]:
url_base = 'http://www.oesz.cn/'
categories = {'睡前故事': url_base + 'shuiqian/',
              '童话故事': url_base + 'tonghua/',
              '寓言故事': url_base + 'yuyan/',
              '成语故事': url_base + 'chengyu/',
              '哲理故事': url_base + 'zheli/',
              '故事大全': url_base + 'gushidaquan/',}

url = categories['睡前故事']
response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'html.parser')

print(soup.prettify())


<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="webkit" name="renderer">
   <meta content="webkit" name="force-rendering"/>
   <meta content="IE=Edge,chrome=1" http-equiv="X-UA-Compatible"/>
   <meta content="width=device-width, initial-scale=1.0, user-scalable=0, minimal-ui" name="viewport"/>
   <meta content="order by www.oesz.cn" name="author"/>
   <meta content="no-transform" http-equiv="Cache-Control">
    <meta content="no-siteapp" http-equiv="Cache-Control"/>
    <meta content="pc,mobile" name="applicable-device"/>
    <meta content="width" name="MobileOptimized"/>
    <meta content="true" name="HandheldFriendly"/>
    <meta content="all" name="Robots"/>
    <title>
     幼儿睡前故事_宝宝睡前小故事_儿童睡前小故事_故事大全网
    </title>
    <meta content="" name="keywords"/>
    <meta content="" name="description"/>
    <link href="/style/css/index.css" rel="stylesheet"/>
    <link href="/style/css/so.css" rel="stylesheet"/>
   </meta>
  </meta>
 </head>
 <body>
  <d

## Step 2.1: Find where stories are stored

```html
 <a href="/shuiqian/11141.html" target="_blank">
    太阳月亮和乌鸦
</a>

<span>
2021-03-31
</span>
```

* `suffix = url.strip('/').split('/')[-1]`: 获取后缀
* `titles = soup.find_all('a', href=...`: 找到所有\<a>并且href以后缀开头的部分
* `titles[i].get_text().strip()`: 获取titles列表里第i个内容的**文字**，并且去除前后空格

In [3]:
suffix = url.strip('/').split('/')[-1]
titles = soup.find_all('a', href=lambda x: x and x.startswith(f'/{suffix}/'))
print(suffix)
titles

shuiqian


[<a href="/shuiqian/">睡前小故事</a>,
 <a href="/shuiqian/" target="_blank">睡前小故事</a>,
 <a href="/shuiqian/11151.html" target="_blank">沙漏的启示_睡前故事</a>,
 <a href="/shuiqian/11150.html" target="_blank">酒爷爷与手套奶奶</a>,
 <a href="/shuiqian/11149.html" target="_blank">南瓜哪儿去了_睡前故事</a>,
 <a href="/shuiqian/11148.html" target="_blank">蜘蛛夫妇的裁缝店</a>,
 <a href="/shuiqian/11147.html" target="_blank">知错能改才能进步</a>,
 <a href="/shuiqian/11146.html" target="_blank">吉米猫分蛋糕_睡前故事</a>,
 <a href="/shuiqian/11145.html" target="_blank">菜园内的故事_睡前故事</a>,
 <a href="/shuiqian/11144.html" target="_blank">会动的房子_睡前故事</a>,
 <a href="/shuiqian/11143.html" target="_blank">狐狸先生的责任</a>,
 <a href="/shuiqian/11142.html" target="_blank">小黑熊打针_睡前故事</a>,
 <a href="/shuiqian/11141.html" target="_blank">太阳月亮和乌鸦</a>,
 <a href="/shuiqian/11140.html" target="_blank">神奇听诊器_睡前故事</a>,
 <a href="/shuiqian/11139.html" target="_blank">割草比赛_睡前故事</a>,
 <a href="/shuiqian/11138.html" target="_blank">同一个朋友_故事网</a>,
 <a href="/shuiqian/11137.html" t

## Step 2.2: Find where indexes are stored
If you didn't see the url to the index (usually number, or "首页", "末页", "下一页") in previous section, locate those indexes. 

```html
<div class="pages">
     <ul>
      <li>
       首页
      </li>
      <li class="thisclass">
       1
      </li>

      ...

       <a href="list_1_2.html">
        下一页
       </a>
      </li>
      <li>
       <a href="list_1_139.html">
        末页
       </a>
      </li>
     </ul>
    </div>
   </div>
  </div>
```


In [4]:
index = soup.find_all('a', href=lambda x: x and x.startswith(f'list_'))
index

[<a href="list_1_2.html">2</a>,
 <a href="list_1_3.html">3</a>,
 <a href="list_1_2.html">下一页</a>,
 <a href="list_1_139.html">末页</a>]

## Step 3: collect story titles in current page
获取本页的所有故事标题和url。

In [5]:
ignore = ['睡前小故事']
category = '睡前故事'
# This store all titles and their links
# format: {'story title': （'url/to/story', 'category') }
titles_dict = {}

for title in titles:
    if title.text.strip() in ignore:
        continue
    titles_dict[title.text.strip()] = url_base + title['href'][1:]


titles_dict['太阳月亮和乌鸦']

'http://www.oesz.cn/shuiqian/11141.html'

定位下一页的url，如果不存在，`next_page = None`。

In [6]:
next_page = None
for i in index:
    if i.get_text().strip() == "下一页":
        next_page = i['href']
        break
next_page = url + next_page
next_page

'http://www.oesz.cn/shuiqian/list_1_2.html'

查看前5个爬取的数据：
* `titles_dict.keys()`：故事标题
* `titles_dict.values()`：故事url

In [7]:
for i in range(5):
    title = list(titles_dict.keys())[i]
    link = titles_dict[title]

    print(f"{title}: {link}")

沙漏的启示_睡前故事: http://www.oesz.cn/shuiqian/11151.html
酒爷爷与手套奶奶: http://www.oesz.cn/shuiqian/11150.html
南瓜哪儿去了_睡前故事: http://www.oesz.cn/shuiqian/11149.html
蜘蛛夫妇的裁缝店: http://www.oesz.cn/shuiqian/11148.html
知错能改才能进步: http://www.oesz.cn/shuiqian/11147.html


### 去除重复部分
我们的爬取策略是找到所有以"/shuiqian"开头的\<a>分区。但网页里“最新发布”区的故事一样符合我们的条件会被爬取。这会造成“最新发布”里的故事被反复爬取

我们的方法是将“最新发布”区的故事名存在一个表中，每次爬取时如果故事名重复则不添加。




```html
 <div class="cont">
 <div class="infohd">
 <div class="info_tab"> <span class="on">最新发布</span> <b class="line"></b> </div>
 </div>
 <div class="index_list clearfix">
 <ul class="clearfix">
 <li><span>2021-03-31</span><a href="/shuiqian/11151.html" target="_blank">沙漏的启示_睡前故事</a></li>
 <li><span>2021-03-31</span><a href="/shuiqian/11150.html" target="_blank">酒爷爷与手套奶奶</a></li>
 ...
```

我们可以找到“最新发布”在`soup`(网页代码)里的位置，然后定位所有在它之后的故事标题。把他们同时加入`recent_dict`和`titles_dict`中。

In [8]:
recent = soup.find('span', string='最新发布').find_all_next('a', href=lambda x: x and x.endswith('.html'))

In [9]:
recent_dict = {}
for title in recent:
    if title.text in ignore:
        continue
    titles_dict[title.text.strip()] = url_base + title['href'][1:]
    recent_dict[title.text.strip()] = url_base + title['href'][1:]

## Step 4: collect story titles in every subsequent pages
获取每一页的标题和网址。
这里用“下一页”是否存在作为代码的结束条件。睡前故事区有139页，每页30个故事链接，所以会有4140~4170个链接。

In [10]:
page_number = 2
while True:
    if page_number % 10 == 0:
        print(f"Page {page_number}")
    ## 重复Step 1-3
    ## Step 1
    response = requests.get(next_page)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    ## Step 2.1
    suffix = url.strip('/').split('/')[-1]
    titles = soup.find_all('a', href=lambda x: x and x.startswith(f'/{suffix}/'))
    ## Step 2.2
    index = soup.find_all('a', href=lambda x: x and x.startswith(f'list_'))

    ## Step 3: crawling the titles and links
    for title in titles:
        ## 如果标题在ignore里或者在recent_dict里，则跳过
        if title.text.strip() in ignore or title.text.strip() in recent_dict.keys():
            continue
        titles_dict[title.text.strip()] = url_base + title['href'][1:]

    next_page = None
    for i in index:
        if i.get_text().strip() == "下一页":
            next_page = i['href']
            break
    
    ## 当下一页为空时, 退出循环
    if not next_page:
        break
        
    next_page = url + next_page
    page_number += 1

print(f"Page {page_number}")

Page 10
Page 20
Page 30
Page 40
Page 50
Page 60
Page 70
Page 80
Page 90
Page 100
Page 110
Page 120
Page 130
Page 139


确认链接的数量是否符合预期

In [11]:
len(titles_dict)

4141

## Step 5: crwaling titles and links for all categories
最后，我们可以执行同样的代码来爬取其他的故事种类的链接，这里我们将爬取第一步`categories`中列出的几种故事：
```python
categories = {'睡前故事': url_base + 'shuiqian/',
              '童话故事': url_base + 'tonghua/',
              '寓言故事': url_base + 'yuyan/',
              '成语故事': url_base + 'chengyu/',
              '哲理故事': url_base + 'zheli/',
              '故事大全': url_base + 'gushidaquan/',}
```

浏览每种故事类型，确认每一种格式相同，处理例外情况


In [12]:
category = '故事大全'

url = categories[category]
response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'html.parser')

suffix = url.strip('/').split('/')[-1]

## 如果是故事大全，则suffix为"a"
if category == '故事大全':
    suffix = 'a'
titles = soup.find_all('a', href=lambda x: x and x.startswith(f'/{suffix}/'))
titles

[<a href="/a/ats/">安徒生童话故事</a>,
 <a href="/a/gl/">格林童话故事</a>,
 <a href="/a/1001/">一千零一夜故事</a>,
 <a href="/a/gongzhu/">公主的故事</a>,
 <a href="/a/xiaogushi/">随意分享小故事</a>,
 <a href="/a/gongzhu/3275.html" target="_blank">月亮公主与男仆</a>,
 <a href="/a/gongzhu/3274.html" target="_blank">画眉嘴国王_【公主童话故事】_ 故事大全阅读频道</a>,
 <a href="/a/gongzhu/3273.html" target="_blank">长发公主历险记</a>,
 <a href="/a/gongzhu/3272.html" target="_blank">蝴蝶公主和千手蜘蛛</a>,
 <a href="/a/gongzhu/3271.html" target="_blank">三个公主的故事</a>,
 <a href="/a/gongzhu/3270.html" target="_blank">玫瑰公主_【公主童话故事】_ 故事大全阅读频道</a>,
 <a href="/a/gongzhu/3269.html" target="_blank">骄做的公主</a>,
 <a href="/a/gongzhu/3268.html" target="_blank">月亮公主_【公主童话故事】_ 故事大全阅读频道</a>,
 <a href="/a/gongzhu/3267.html" target="_blank">龙公主戏神珠</a>,
 <a href="/a/gongzhu/3266.html" target="_blank">笑口难开的公主</a>,
 <a href="/a/gongzhu/3265.html" target="_blank">公主的珍珠鞋_民间故事_童话故事网</a>,
 <a href="/a/gongzhu/3264.html" target="_blank">脱隆科兰公主_【公主童话故事】_ 故事大全阅读频道</a>,
 <a href="/a/gongzhu/3263

将不需要的链接加入`ignore`。

In [13]:
ignore = ['睡前小故事', '童话故事', '寓言故事', '成语故事', '哲理故事', '安徒生童话故事', '格林童话故事', '一千零一夜故事', '公主的故事', '随意分享小故事', '故事大全']


在单个故事种类上测试：

In [14]:
## 除了每个故事的链接，还需要保存每个故事的分类
## 所以格式为: {'story title': （'url/to/story', 'category') }
titles_dict = {}
ignore = ['睡前小故事', '童话故事', '寓言故事', '成语故事', '哲理故事', '安徒生童话故事', '格林童话故事', '一千零一夜故事', '公主的故事', '随意分享小故事', '故事大全']

# for ...
category = '哲理故事'

url = categories[category]
response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'html.parser')

suffix = url.strip('/').split('/')[-1]

## 如果是故事大全，则suffix为"a"
if category == '故事大全':
    suffix = 'a'
titles = soup.find_all('a', href=lambda x: x and x.startswith(f'/{suffix}/'))

index = soup.find_all('a', href=lambda x: x and x.startswith(f'list_'))

## 爬取标题和链接
for title in titles:
    if title.text.strip() in ignore:
        continue
    titles_dict[title.text.strip()] = (url_base + title['href'][1:], category)

## 爬取下一页链接
next_page = None
for i in index:
    if i.get_text().strip() == "下一页":
        next_page = i['href']
        break
next_page = url + next_page

# 如果最新发布区的故事已经在titles_dict里，则不再重复添加
if not recent_dict:
    print("No recent stories")
    recent = soup.find('span', string='最新发布').find_all_next('a', href=lambda x: x and x.endswith('.html'))
    recent_dict = {}
    for title in recent:
        recent_suffix = title['href'].split('/')[-2]
        if title.text in ignore or recent_suffix != suffix:
            continue
        titles_dict[title.text.strip()] = (url_base + title['href'][1:], category)
        recent_dict[title.text.strip()] = (url_base + title['href'][1:], category)

## Step 4: 循环爬取下一页
page_number = 2
while True:
    if page_number % 10 == 0:
        print(f"Page {page_number}")
    ## 重复Step 1-3
    ## Step 1
    response = requests.get(next_page)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    ## Step 2.1
    suffix = url.strip('/').split('/')[-1]
    titles = soup.find_all('a', href=lambda x: x and x.startswith(f'/{suffix}/'))
    ## Step 2.2
    index = soup.find_all('a', href=lambda x: x and x.startswith(f'list_'))

    ## Step 3: crawling the titles and links
    for title in titles:
        ## 如果标题在ignore里或者在recent_dict里，则跳过
        if title.text.strip() in ignore or title.text.strip() in recent_dict.keys():
            continue
        titles_dict[title.text.strip()] = (url_base + title['href'][1:], category)

    next_page = None
    for i in index:
        if i.get_text().strip() == "下一页":
            next_page = i['href']
            break
    
    ## 当下一页为空时, 退出循环
    if not next_page:
        break
        
    next_page = url + next_page
    page_number += 1

print(f"Page {page_number}")


Page 10
Page 11


加入循环，爬取所有种类故事的标题和链接：

In [15]:
## 除了每个故事的链接，还需要保存每个故事的分类
## 所以格式为: {'story title': （'url/to/story', 'category') }
titles_dict = {}
ignore = ['睡前小故事', '童话故事', '寓言故事', '成语故事', '哲理故事', '安徒生童话故事', '格林童话故事', '一千零一夜故事', '公主的故事', '随意分享小故事', '故事大全']

for category in categories:
    print(f"Category: {category}")
    
    url = categories[category]
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')

    suffix = url.strip('/').split('/')[-1]

    ## 如果是故事大全，则suffix为"a"
    if category == '故事大全':
        suffix = 'a'
    titles = soup.find_all('a', href=lambda x: x and x.startswith(f'/{suffix}/'))

    index = soup.find_all('a', href=lambda x: x and x.startswith(f'list_'))

    ## 爬取标题和链接
    for title in titles:
        if title.text.strip() in ignore:
            continue
        titles_dict[title.text.strip()] = (url_base + title['href'][1:], category)

    ## 爬取下一页链接
    next_page = None
    for i in index:
        if i.get_text().strip() == "下一页":
            next_page = i['href']
            break
    next_page = url + next_page

    # 如果最新发布区的故事已经在titles_dict里，则不再重复添加
    if not recent_dict:
        print("No recent stories")
        recent = soup.find('span', string='最新发布').find_all_next('a', href=lambda x: x and x.endswith('.html'))
        recent_dict = {}
        for title in recent:
            recent_suffix = title['href'].split('/')[-2]
            if title.text in ignore or recent_suffix != suffix:
                continue
            titles_dict[title.text.strip()] = (url_base + title['href'][1:], category)
            recent_dict[title.text.strip()] = (url_base + title['href'][1:], category)

    ## Step 4: 循环爬取下一页
    page_number = 2
    while True:
        if page_number % 10 == 0:
            print(f"Page {page_number}")
        ## 重复Step 1-3
        ## Step 1
        response = requests.get(next_page)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')
        ## Step 2.1
        suffix = url.strip('/').split('/')[-1]
        titles = soup.find_all('a', href=lambda x: x and x.startswith(f'/{suffix}/'))
        ## Step 2.2
        index = soup.find_all('a', href=lambda x: x and x.startswith(f'list_'))

        ## Step 3: crawling the titles and links
        for title in titles:
            ## 如果标题在ignore里或者在recent_dict里，则跳过
            if title.text.strip() in ignore or title.text.strip() in recent_dict.keys():
                continue
            titles_dict[title.text.strip()] = (url_base + title['href'][1:], category)

        next_page = None
        for i in index:
            if i.get_text().strip() == "下一页":
                next_page = i['href']
                break
        
        ## 当下一页为空时, 退出循环
        if not next_page:
            break
            
        next_page = url + next_page
        page_number += 1

    print(f"Page {page_number}")


Category: 睡前故事
Page 10
Page 20
Page 30
Page 40
Page 50
Page 60
Page 70
Page 80
Page 90
Page 100
Page 110
Page 120
Page 130
Page 139
Category: 童话故事
Page 4
Category: 寓言故事
Page 10
Page 20
Page 30
Page 40
Page 50
Page 57
Category: 成语故事
Page 10
Page 20
Page 30
Page 40
Page 50
Page 60
Page 61
Category: 哲理故事
Page 10
Page 11
Category: 故事大全
Page 7


检查爬取到的链接总数：

In [16]:
len(titles_dict)

8116

## Final Step: Crawl the stories
最后一步是通过链接获取故事的内容。我们的链接储存在`titles_dict`中，格式为`titles_dict['故事名字'] = (故事链接, 故事种类)`。


In [17]:
titles_dict['梨子和苹果_寓言故事']

('http://www.oesz.cn/yuyan/3867.html', '寓言故事')

我们使用`pandas`的`DataFrame`格式来储存数据。这是一种类似excel表格的，有行和列的格式，因此可以直接保存为便于打开的`csv`格式。

首先将已有的标题和url读入`DataFrame`:

In [18]:
data = [(name, info[0], info[1]) for name, info in titles_dict.items()]
df = pd.DataFrame(data, columns=['name', 'url', 'category'])

print(f"number of rows: {df.shape[0]}\nnumber of columns: {df.shape[1]}")

df.head()


number of rows: 8116
number of columns: 3


,name,url,category
0,沙漏的启示_睡前故事,http://www.oesz.cn/shuiqian/11151.html,睡前故事
1,酒爷爷与手套奶奶,http://www.oesz.cn/shuiqian/11150.html,睡前故事
2,南瓜哪儿去了_睡前故事,http://www.oesz.cn/shuiqian/11149.html,睡前故事
3,蜘蛛夫妇的裁缝店,http://www.oesz.cn/shuiqian/11148.html,睡前故事
4,知错能改才能进步,http://www.oesz.cn/shuiqian/11147.html,睡前故事


### 获取故事内容
通过检查网址代码可以发现故事储存在`\<div>`下，并且标注`class="content"`。
```html
<div class="content">
    <p>
    <p>
        小男孩弗兰克非常健谈，如果他发现件新鲜东西，肯定会提出许多许多的问题来。他的母亲总是不厌其烦地、耐心和蔼地，尽力回答可以回答的问题。
    </p>
    <p>
        偶尔，她也会在这样说：“孩子，现在你还小，到你长大到十岁的时候，再来问我这个问题怎么样?到时候我会详细地告诉你，现在说了你也不理解。”
    </p>
    ...
```

**代码解释**：
```python
soup.find('div', class_='content').text.rstrip().replace('\r', '').replace('\u3000', '')
```
* `soup.find('div', class_='content')`：找到`\<div>`下，并且标注`class="content"`的内容
* `.text`：获取文字
* `rstrip().replace('\r', '').replace('\u3000', '')`：去除多余字符

In [75]:
url = df.url[0]
response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'html.parser')

print(soup.find('div', class_='content').text.rstrip().replace('\r', '').replace('\u3000', ''))




	小男孩弗兰克非常健谈，如果他发现件新鲜东西，肯定会提出许多许多的问题来。他的母亲总是不厌其烦地、耐心和蔼地，尽力回答可以回答的问题。

	偶尔，她也会在这样说：“孩子，现在你还小，到你长大到十岁的时候，再来问我这个问题怎么样?到时候我会详细地告诉你，现在说了你也不理解。”

	每当这个时候，弗兰克便会放下他的问题，不再强求了。因为他知道，只要提出的问题合适，妈妈就一定会认真地回答他的。

	母亲说：“按 8字外形制成沙漏，是古代的一种计时器。沙从一端倒人后，便流过中间的小孔。至于往里面装多少沙子，那要看一个小时沙子从小孔漏过的数量来定了。”

	那细细的沙流流得好慢，童话故事，弗兰克看着看着，不耐烦了。“妈妈，让我帮助它摇一摇吧。”他着急地说道，“流得这么慢了，可没有时间流完了。”

	“对，孩子，就是这样的，”母亲说：“这沙子虽然流动得很慢，却没有停止流动。你观察钟表上的指针，是否发现也走得极其慢。是啊，的确如此，不过，它们并没停下来过。孩子，不管你是在玩还是睡觉，这些沙子依然在一粒粒地流动，钟表指针在一秒接一秒地

	止动。整个晚上，时钟上的时针，将转整个一圈，沙漏中的沙子，将流经小孔12次。因为，它们从未停下工作，它们不愿停下自己的脚步，去过多思虑该做多少。”

	听从了母亲的教导，弗兰克真的非常勤奋地一行一行地学了起来。几小时过去后，他真得懂得了那赞美诗的含义啦。沙漏的启示_睡前故事


In [78]:
df.loc['content'] = None
for i in range(df.shape[0]):
    if i % 20 == 0 and i != 0:
        print(f"{i} Stories processed")
        break
    url = df.url[i]
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    ## 从网页中提取故事内容
    content = soup.find('div', class_='content').text.rstrip().replace('\r', '').replace('\u3000', '')
    df.loc[i, 'content'] = content

20 Stories processed


In [79]:
df.to_csv('stories_cn_oesz.csv', index=False)

,name,url,category,content
0,沙漏的启示_睡前故事,http://www.oesz.cn/shuiqian/11151.html,睡前故事,\n\n\t小男孩弗兰克非常健谈，如果他发现件新鲜东西，肯定会提出许多许多的问题来。他的母亲...
1,酒爷爷与手套奶奶,http://www.oesz.cn/shuiqian/11150.html,睡前故事,\n\n\t酒爷爷与手套奶奶酒爷爷喝醉了酒，在路上抱了一只小狗回家。醒来后，酒爷爷看见一只小...
2,南瓜哪儿去了_睡前故事,http://www.oesz.cn/shuiqian/11149.html,睡前故事,\n\n\t南瓜哪儿去了_睡前故事“南瓜?”小兔子说，“请原谅我，我不知道南瓜是你家的，它就...
3,蜘蛛夫妇的裁缝店,http://www.oesz.cn/shuiqian/11148.html,睡前故事,\n\n\t蜘蛛夫妇的裁缝店蜘蛛先生发现了一个商机，它决定和他的夫人一起开一家裁缝店，蜘蛛太...
4,知错能改才能进步,http://www.oesz.cn/shuiqian/11147.html,睡前故事,\n\n\t小猫咪咪看着太阳好，于是在外面玩，但是它抓不到蝴蝶，于是看到蜻蜓抓到蝴蝶心里很不...
...,...,...,...,...
8112,白雪公主,http://www.oesz.cn/a/gongzhu/854.html,故事大全,NaN
8113,骄做的公主_【公主童话故事】_ 故事大全阅读频道,http://www.oesz.cn/a/gongzhu/853.html,故事大全,NaN
8114,苦命姑娘_【公主童话故事】_ 故事大全阅读频道,http://www.oesz.cn/a/gongzhu/852.html,故事大全,NaN
8115,通济桥头公主泪,http://www.oesz.cn/a/gongzhu/851.html,故事大全,NaN
